In [ ]:
%matplotlib notebook
#%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import rasterio
import rasterio.warp
import numpy as np
import math

import pyproj
import xarray as xr
import fiona
import shapely
import pydash
from osgeo import osr
from numpy import pi, r_, c_, vstack, hstack
import sys

from kk.dggs import DGGS
from kk.tools import polygon_path

In [ ]:
dg = DGGS()

In [ ]:
xx, yy = np.meshgrid(np.arange(-180,180,1),
                     np.arange(-90,90,1))
xx = xx.ravel()
yy = yy.ravel()
xx.shape, yy.shape

In [ ]:
def gen_test_plots(dg, xx, yy, scale_level = 8):
    idx, x, y = dg.to_ixy(xx, yy, 8)
    u,v = dg._rh(xx,yy)
    aa = dg.to_address(xx, yy, 8)

    plt.figure(figsize=(9,2))
    plt.plot(idx)
    
    plt.figure(figsize=(9,4))
    plt.scatter(u,v, c=idx, marker='.')

    print(set(DGGS.i2n[idx])),
    print(set(a for a in (aa>>60)&7))
    

gen_test_plots(dg, xx, yy, 15)

In [ ]:
def check_scale(cell_code, level, plot = True):
    cell_addr = cell_code + '0'*level
    W = 3**level
    M = W - 1
    S = min(W, 256)
    
    tr, (maxW, maxH) = dg.pixel_coord_transform(cell_addr)
    b = 0.5 - 1e-6
    
    print('max.dim: {}x{}'.format(maxW, maxH))
    
    print('TL: {:13.8f} {:13.8f}'.format(*tr(-b , -b)))
    print('TR: {:13.8f} {:13.8f}'.format(*tr(M+b, -b)))
    print('BR: {:13.8f} {:13.8f}'.format(*tr(M+b,M+b)))
    print('BL: {:13.8f} {:13.8f}'.format(*tr(-b ,M+b)))
    
    p = np.linspace(0, M, S).astype('uint32')
    px,py = [a.ravel() for a in np.meshgrid(p, p)]
    u,v = tr(px,py)

    if plot:
        plt.figure(figsize=(9,4))
        plt.subplot(121)
        plt.imshow(u.reshape(S,S))
        plt.colorbar()
        plt.subplot(122)
        plt.imshow(v.reshape(S,S))
        plt.colorbar() 
    
    

check_scale('S', 15)
check_scale('R', 15)

In [ ]:
for s in 'SNOPQR':
    print(s)
    check_scale(s, 15, plot=False)

In [ ]:
def add_cell_plot(cell_addr, style='-', ax=None):
    tr, (maxW, maxH) = dg.pixel_coord_transform(cell_addr)

    bb = 0.5 - 1e-8
    x = np.linspace(-bb, maxW - 1 + bb, 2)
    y = np.linspace(-bb, maxH - 1 + bb, 2)
    x,y = polygon_path(x,y)

    u,v = tr(x,y)
    
    ax.plot(u,v, style)
    
    
fig = plt.figure(figsize=(8,8))
ax = fig.add_subplot(111)
add_cell_plot('R000', 'r.-', ax)
add_cell_plot('R431', 'b.-', ax)
add_cell_plot('R181', 'g.-', ax)
add_cell_plot('R882', 'c.-', ax)
add_cell_plot('R888', 'm.-', ax)
ax.axis('equal')

pass

-------------------------------------------------------------------